# Перевод из PDF в текст или просто берем текст

In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.3 MB/s eta 0:00:00


In [31]:
# pdf_reader.py
import pdfplumber
from pathlib import Path


def pdf_file_to_text(pdf_path):
    pdf_path = Path(pdf_path)
    if not pdf_path.is_file():
        raise FileNotFoundError(f"PDF-файл не найден: {pdf_path}")

    pages_text = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            pages_text.append(page_text.strip())

    text = "\n\n".join(pages_text).strip()
    return text


# Classification model

In [32]:
import zipfile

zip_path = "/content/rubert-letter-classifier-final.zip"
extract_to = "/content"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_to)

In [33]:
# classifier.py
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_PATH = "./rubert-letter-classifier-final"
BASE_MODEL = "DeepPavlov/rubert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

clf_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH
).to(DEVICE)

ID2LABEL = clf_model.config.id2label


def classify_letter(text):
    inputs = tokenizer(
        text,
        truncation=True,
        max_length=256,
        padding="max_length",
        return_tensors="pt"
    ).to(DEVICE)

    with torch.no_grad():
        logits = clf_model(**inputs).logits
        probs = torch.softmax(logits, dim=-1)[0].cpu().numpy()

    best_idx = int(probs.argmax())
    return {
        "type": ID2LABEL[best_idx],
        "confidence": float(probs[best_idx]),
        "probs": {ID2LABEL[i]: float(p) for i, p in enumerate(probs)}
    }


# LLM для summary+выделние ключевых признаков, а затем генерации ответного письма

In [34]:
!wget https://storage.yandexcloud.net/ycpub/maikeys/.env

# yandex_llm_client.py
from dotenv import load_dotenv
import os
import json
from typing import Dict, List, Optional
from openai import OpenAI

load_dotenv(".env")

FOLDER_ID = os.getenv("folder_id")
API_KEY = os.getenv("api_key")

if not FOLDER_ID or not API_KEY:
    raise RuntimeError(
        "Не найдены переменные окружения 'folder_id' и/или 'api_key'. "
        "Добавьте их в .env или через переменные окружения."
    )

BASE_URL = "https://rest-assistant.api.cloud.yandex.net/v1"


MODEL = f"gpt://{FOLDER_ID}/yandexgpt-lite/latest"

client = OpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
    project=FOLDER_ID,
)


def yagpt_lite_raw(
    text,
    system_prompt = "Ты — полезный ассистент.",
    temperature = 0.2,
    max_tokens = 500,
):
    response = client.responses.create(
        model=MODEL,
        instructions=system_prompt,
        input=text,
        temperature=temperature,
        max_output_tokens=max_tokens,
    )

    return response.output_text.strip()

--2025-11-29 00:38:01--  https://storage.yandexcloud.net/ycpub/maikeys/.env
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80 [application/x-www-form-urlencoded]
Saving to: ‘.env.6’

.env.6              100%[===================>]      80  --.-KB/s    in 0s      

2025-11-29 00:38:02 (21.1 MB/s) - ‘.env.6’ saved [80/80]



## summary+ключевые признаки

In [35]:
def yagpt_summary(email_text):
    system_prompt = """
    Ты — помощник в юридическом департаменте банка.
    Твоя задача — кратко пересказать содержание входящего письма.

    Требования:
    - Пиши на русском языке.
    - Используй 2–4 предложения.
    - Отрази: кто пишет, по какому вопросу, чего ожидает от Банка.
    - Не добавляй новую информацию, которой нет в письме.
    - Не давай рекомендаций и не отвечай от лица Банка.
    - Пиши в нейтральном деловом стиле, как аналитическая выжимка.
    """

    user_text = (
        "Ниже приведён текст письма. Сделай краткое резюме по указанным выше правилам.\n\n"
        f"Текст письма:\n\"\"\"\n{email_text}\n\"\"\"\n\n"
        "Краткое резюме:"
    )

    return yagpt_lite_raw(
        text=user_text,
        system_prompt=system_prompt,
        temperature=0.1,
        max_tokens=250,
    )

In [36]:
def yagpt_extract_facts(email_text):
    system_prompt = """
    Ты — ассистент в юридическом департаменте банка.
    Твоя задача — структурировать информацию из входящего письма.

    ВАЖНО:
    - Верни строго JSON-объект.
    - Не добавляй лишний текст до или после JSON.
    - Если каких-то данных нет в письме, верни пустой список или пустую строку.
    Формат:
    {
      "organizations": [...],
      "dates": [...],
      "doc_numbers": [...],
      "requirements": "...",
      "contacts": {
        "emails": [...],
        "phones": [...],
        "person_names": [...]
      },
      "requisites": {
        "inn": "...",
        "kpp": "...",
        "ogrn": "...",
        "account_numbers": [...]
      },
      "regulatory_refs": [...]
    }
    """

    user_text = (
        "Разбери текст письма и выдели ключевые признаки по следующей схеме.\n"
        "- organizations: организации/компании (ООО, АО, ПАО и т.п.).\n"
        "- dates: важные даты или диапазоны дат.\n"
        "- doc_numbers: номера договоров, счетов, обращений (обычно с символом № или похожим).\n"
        "- requirements: одной строкой сформулируй, чего хочет автор от Банка.\n"
        "- contacts.emails: все email-адреса.\n"
        "- contacts.phones: телефоны.\n"
        "- contacts.person_names: ФИО людей, если есть.\n"
        "- requisites.inn, kpp, ogrn: реквизиты организации, если указаны.\n"
        "- requisites.account_numbers: номера счетов.\n"
        "- regulatory_refs: упомянутые законы, нормативные акты, положения ЦБ и т.п.\n\n"
        f"Текст письма:\n\"\"\"\n{email_text}\n\"\"\"\n\n"
        "Верни только JSON в указанном выше формате."
    )

    raw = yagpt_lite_raw(
        text=user_text,
        system_prompt=system_prompt,
        temperature=0.0,
        max_tokens=600,
    )

    try:
        facts = json.loads(raw)
    except json.JSONDecodeError:
        try:
            start = raw.index("{")
            end = raw.rindex("}") + 1
            json_str = raw[start:end]
            facts = json.loads(json_str)
        except Exception:
            facts = {}

    if not isinstance(facts, dict):
        facts = {}

    # Базовые поля
    facts.setdefault("organizations", [])
    facts.setdefault("dates", [])
    facts.setdefault("doc_numbers", [])
    facts.setdefault("requirements", "")

    # Контакты
    contacts = facts.get("contacts") or {}
    if not isinstance(contacts, dict):
        contacts = {}
    contacts.setdefault("emails", [])
    contacts.setdefault("phones", [])
    contacts.setdefault("person_names", [])
    facts["contacts"] = contacts

    # Реквизиты
    requisites = facts.get("requisites") or {}
    if not isinstance(requisites, dict):
        requisites = {}
    requisites.setdefault("inn", "")
    requisites.setdefault("kpp", "")
    requisites.setdefault("ogrn", "")
    requisites.setdefault("account_numbers", [])
    facts["requisites"] = requisites

    # Нормативные ссылки
    facts.setdefault("regulatory_refs", [])

    # Нормализация типов
    if not isinstance(facts["organizations"], list):
        facts["organizations"] = [str(facts["organizations"])]
    if not isinstance(facts["dates"], list):
        facts["dates"] = [str(facts["dates"])]
    if not isinstance(facts["doc_numbers"], list):
        facts["doc_numbers"] = [str(facts["doc_numbers"])]
    if not isinstance(facts["requirements"], str):
        facts["requirements"] = str(facts["requirements"])

    for key in ["emails", "phones", "person_names"]:
        val = facts["contacts"].get(key)
        if not isinstance(val, list):
            facts["contacts"][key] = [str(val)] if val else []

    if not isinstance(facts["requisites"]["account_numbers"], list):
        acc = facts["requisites"]["account_numbers"]
        facts["requisites"]["account_numbers"] = [str(acc)] if acc else []

    if not isinstance(facts["regulatory_refs"], list):
        facts["regulatory_refs"] = [str(facts["regulatory_refs"])]

    return facts

## генерация ответа

In [37]:
def yagpt_reply(
    email_text,
    letter_type,
    facts = None,
    style = "official",
    history = None,
):

    class_instr = f"Тип письма (по классификатору): {letter_type}.\n"

    lt = letter_type.lower()
    if "жалоб" in lt or "претенз" in lt:
        class_instr += (
            "Это жалоба или претензия. В ответе важно поблагодарить за обратную связь, "
            "корректно отреагировать на недовольство, описать дальнейшие шаги, "
            "но не признавать юридическую вину.\n"
        )
    elif "запрос" in lt and "информац" in lt:
        class_instr += (
            "Это запрос информации или документов. В ответе нужно указать, какую информацию банк предоставит, "
            "в какие сроки и какие дополнительные данные могут потребоваться.\n"
        )
    elif "регулятор" in lt:
        class_instr += (
            "Это регуляторный запрос. Ответ должен быть максимально формальным, "
            "юридически выверенным и аккуратным.\n"
        )
    elif "партнер" in lt or "партнёр" in lt:
        class_instr += (
            "Это партнёрское предложение. Ответ должен быть вежливым, подчеркнуть интерес или корректно отказать, "
            "и предложить возможные дальнейшие шаги.\n"
        )
    elif "согласован" in lt:
        class_instr += (
            "Это запрос на согласование. В ответе нужно явно указать результат согласования "
            "(согласовано / требуется доработка / частично согласовано) и следующие шаги.\n"
        )
    elif "уведомлен" in lt or "уведомление" in lt:
        class_instr += (
            "Это уведомление. Ответ может быть кратким подтверждением получения и описанием дальнейших действий.\n"
        )

    if style == "official":
        style_instr = "Стиль: строго официальный, формальный, без лишних эмоций.\n"
    elif style == "corporate":
        style_instr = "Стиль: стандартный корпоративный деловой, нейтральный.\n"
    elif style == "client":
        style_instr = (
            "Стиль: клиентоориентированный, вежливый, более тёплый, но при этом официальный.\n"
        )
    elif style == "short":
        style_instr = (
            "Стиль: краткий информационный ответ. "
            "Сформулируй только ключевую информацию и базовые формулировки вежливости, без деталей.\n"
        )
    else:
        style_instr = "Стиль: стандартный корпоративный деловой.\n"

    if facts:
        orgs = ", ".join(facts.get("organizations", [])) or "—"
        dates = ", ".join(facts.get("dates", [])) or "—"
        docs = ", ".join(facts.get("doc_numbers", [])) or "—"
        req = facts.get("requirements") or "—"
        regs = ", ".join(facts.get("regulatory_refs", [])) or "—"

        contacts = facts.get("contacts", {})
        emails = ", ".join(contacts.get("emails", [])) or "—"
        phones = ", ".join(contacts.get("phones", [])) or "—"
        persons = ", ".join(contacts.get("person_names", [])) or "—"

        requisites = facts.get("requisites", {})
        inn = requisites.get("inn") or "—"
        kpp = requisites.get("kpp") or "—"
        ogrn = requisites.get("ogrn") or "—"
        accs = ", ".join(requisites.get("account_numbers", [])) or "—"

        facts_str = (
            "Выделенные факты:\n"
            f"- Организации: {orgs}\n"
            f"- Даты: {dates}\n"
            f"- Документы/договора: {docs}\n"
            f"- Требование/ожидание: {req}\n"
            f"- Нормативные акты: {regs}\n"
            f"- Контакты (email): {emails}\n"
            f"- Контакты (телефон): {phones}\n"
            f"- Представители/ФИО: {persons}\n"
            f"- ИНН: {inn}, КПП: {kpp}, ОГРН: {ogrn}\n"
            f"- Номера счетов: {accs}\n"
        )
    else:
        facts_str = "Выделенные факты: не удалось определить.\n"

    history_block = ""
    if history:
        if isinstance(history, list):
            joined = "\n---\n".join(history)
        else:
            joined = str(history)
        history_block = (
            "История предыдущей переписки с этим отправителем:\n"
            f"{joined}\n\n"
            "Учитывай эту историю при формировании ответа, но не переписывай её дословно.\n"
        )

    system_prompt = """
    Ты — сотрудник службы клиентской поддержки крупного банка.
    Ты готовишь официальный текст ответа на входящие письма клиентов, партнёров и регуляторов.

    Требования:
    - Не придумывай факты о продуктах, тарифах, договорах и нормативных актах.
    - Если каких-то данных явно нет в письме, напиши, что требуется дополнительная информация
      или что вопрос будет передан в профильное подразделение.
    - Соблюдай нейтральный, корректный и вежливый тон даже при жалобах и претензиях.
    - Не используй квадратные скобки вида [текст]. Если информации не хватает — используй нейтральные
      формулировки вроде «сотрудник Банка», «дата получения письма», «Служба клиентской поддержки Банка».
    - Ответ должен быть готовым текстом письма, который можно отправить от имени Банка.
    """

    user_text = (
        f"{class_instr}"
        f"{style_instr}\n"
        f"{history_block}"
        f"{facts_str}\n"
        f"Текст исходного письма:\n\"\"\"\n{email_text}\n\"\"\"\n\n"
        "Сформируй текст ответа (обращение, основной текст, финальная формула вежливости)."
    )

    return yagpt_lite_raw(
        text=user_text,
        system_prompt=system_prompt,
        temperature=0.3 if style != "short" else 0.1,
        max_tokens=500 if style != "short" else 200,
    )


In [38]:
def yagpt_generate_all_styles(
    email_text,
    letter_type,
    facts = None,
    history = None,
):
    styles = ["official", "corporate", "client"]
    replies = {}

    for st in styles:
        replies[st] = yagpt_reply(
            email_text=email_text,
            letter_type=letter_type,
            facts=facts,
            style=st,
            history=history,
        )

    return replies


# Pipeline

In [43]:
#from classifier import classify_letter
#from pdf_reader import pdf_file_to_text
#from yandex_llm_client import (
#    yagpt_summary,
#    yagpt_extract_facts,
#    yagpt_reply,
#    yagpt_generate_all_styles
#)


def process_email(email_text, style = "official"):
    # 1. Классификация (ruBERT)
    clf = classify_letter(email_text)
    letter_type = clf["type"]
    confidence = clf["confidence"]

    # 2. Ключевые признаки (LLM -> JSON)
    facts = yagpt_extract_facts(email_text)

    # 3. Summary письма (LLM)
    summary = yagpt_summary(email_text)

    # 4. Генерация ответа (LLM)
    replies = yagpt_generate_all_styles(
    email_text=email_text,
    letter_type=letter_type,
    facts=facts,
    history=None
    )

    return {
    "letter_type": letter_type,
    "type_confidence": confidence,
    "facts": facts,
    "summary": summary,
    "replies": replies
    }



def process_email_from_pdf(pdf_path, style = "official"):
    text = pdf_file_to_text(pdf_path)
    return process_email(text, style=style)

# Demo

In [44]:
# demo.py
#from pipeline import process_email, process_email_from_pdf


def demo_text():
    example_email = """
    Уважаемые коллеги!

    Компания ООО «ТехПроект» обращается с просьбой предоставить
    выписку по расчетному счёту 40802810000000012345 за период
    с 01.09.2024 по 30.09.2024, а также копии договоров по РКО.
    Документы необходимы для предоставления в налоговый орган.
    Просим направить их не позднее 10.10.2024.

    С уважением,
    ООО «ТехПроект»
    """

    result = process_email(example_email, style="corporate")

    print("ПРИМЕР 1: ТЕКСТОВОЕ ПИСЬМО\n")

    print("Предсказанный класс письма(ruBERT):")
    print(f"  {result['letter_type']} ")

    print("\n Summary исходного письма (YandexGPT-Lite):\n")
    print(result["summary"])

    print("\n Ключевые признаки:")
    for k, v in result["facts"].items():
        print(f"  {k}: {v}")

    print("\n Варианты ответов \n")
    for style, text in result["replies"].items():
      print(f"[{style.upper()}]\n{text}\n{'-'*60}\n")



def demo_pdf():
    pdf_path = "/content/test_letter.pdf"

    try:
        result = process_email_from_pdf(pdf_path, style="official")
    except FileNotFoundError:
        print(f"PDF-файл '{pdf_path}' не найден. Пропускаем демо с PDF.")
        return

    print("ПРИМЕР 2: ПИСЬМО ИЗ PDF \n")

    print("Предсказанный класс письма (ruBERT):")
    print(f"  {result['letter_type']}")

    print("\n Summary исходного письма (YandexGPT-Lite):\n")
    print(result["summary"])

    print("\n Ключевые признаки:")
    for k, v in result["facts"].items():
        print(f"  {k}: {v}")

    print("\n Варианты ответов\n")
    for style, text in result["replies"].items():
      print(f"[{style.upper()}]\n{text}\n{'-'*60}\n")


if __name__ == "__main__":
    demo_text()
    #demo_pdf()


ПРИМЕР 1: ТЕКСТОВОЕ ПИСЬМО

Предсказанный класс письма(ruBERT):
  Уведомление или информирование 

 Summary исходного письма (YandexGPT-Lite):

Компания ООО «ТехПроект» просит предоставить выписку по расчётному счёту № 40802810000000012345 за период с 01.09.2024 по 30.09.2024 и копии договоров по РКО. Документы нужны для предоставления в налоговый орган. Срок — не позднее 10.10.2024.

 Ключевые признаки:
  organizations: ['ООО «ТехПроект»']
  dates: ['01.09.2024', '30.09.2024', '10.10.2024']
  doc_numbers: ['40802810000000012345']
  requirements: предоставить выписку по расчётному счёту за указанный период и копии договоров по РКО
  contacts: {'emails': [], 'phones': [], 'person_names': []}
  requisites: {'inn': '', 'kpp': '', 'ogrn': '', 'account_numbers': ['40802810000000012345']}
  regulatory_refs: []

 Варианты ответов 

[OFFICIAL]
Уважаемый представитель ООО «ТехПроект»!

Ваше обращение от [дата получения письма] получено. В ответ на вашу просьбу Служба клиентской поддержки Банка 

In [30]:
demo_pdf()

=== ПРИМЕР 2: ПИСЬМО ИЗ PDF ===

→ Предсказанный класс письма (ruBERT):
  Запрос на согласование | confidence = 0.262

→ Summary исходного письма (YandexGPT-Lite):

Автор письма уведомляет о нарушении условий договора №БС-1456 от 15.03.2023, ссылаясь на списание средств без предварительного уведомления. Требует немедленного разъяснения и возврата средств в течение 3 рабочих дней.

→ Ключевые признаки (YandexGPT-Lite → JSON):
  organizations: ['неизвестно']
  dates: ['15.03.2023']
  doc_numbers: ['БС-1456']
  requirements: Немедленного разъяснения и возврата средств в течение 3 рабочих дней.
  contacts: {'emails': [], 'phones': [], 'person_names': []}
  requisites: {'inn': '', 'kpp': '', 'ogrn': '', 'account_numbers': []}
  regulatory_refs: []

=== Варианты ответов (3 стиля) ===

[OFFICIAL]
Уважаемый клиент,

Ваше обращение от 15.03.2023 по договору №БС-1456 рассмотрено. Для уточнения всех обстоятельств и принятия решения по вашему запросу требуется дополнительная информация.

Просим ва